In [ ]:
#papermill_description=imports

import json
import os
import geopandas as gpd
from io import StringIO
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Configure logging level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
#papermill_description=openmeteo_api_setup

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)
